In [1]:
import pandas as pd
import numpy as np
import torch
from skimage import io

In [29]:
df_male = pd.read_csv("./male_yellow_images.csv")
df_female = pd.read_csv("./female_yellow_images.csv")
df_mtw = pd.read_csv("./male_white_images.csv")
df_ftw = pd.read_csv("./female_white_images.csv")

In [3]:
df_male.head()

,Email,Filename,Rating,original Rating
0,1,mty1.jpg,4,NaN
1,2,mty1.jpg,4,4.0
2,3,mty1.jpg,5,NaN
3,4,mty1.jpg,3,NaN
4,5,mty1.jpg,4,NaN


In [4]:
df_male.values.shape

(120000, 4)

In [7]:
def get_median(num_people, df_people, path, prefix):
    target_columns = ['Filename', 'Rating', 'Female', 'Male', 'Yellow','White']
    df_rt = pd.DataFrame(columns = target_columns)
    for i in range(num_people):
        filename = "%s%d.jpg"%(prefix, i+1)
#         rating_mean = int(df_people.loc[df_people['Filename'] == filename, 'Rating'].mean() + 0.49) - 1
        rating_mean = df_people.loc[df_people['Filename'] == filename, 'Rating'].mean()
        male = 1 if prefix[0] == 'm' else 0
        female = 1 if prefix[0] == 'f' else 0
        yellow = 1 if prefix[2] == 'y' else 0
        white = 1 if prefix[2] == 'w' else 0
#         landmark = list(landmarks['./Images/'+filename].view(-1).numpy().astype(np.int))
        row = pd.Series([path + filename, rating_mean, female, male, yellow, white], index=target_columns)
        df_rt = df_rt.append(row, ignore_index=True)
    return df_rt

In [8]:
num_male = 2000
df_mty = get_median(num_male, df_male, "./ImageFolder/", "mty")

In [9]:
df_mty.head()

,Filename,Rating,Female,Male,Yellow,White
0,./ImageFolder/mty1.jpg,3.950000,0,1,1,0
1,./ImageFolder/mty2.jpg,3.600000,0,1,1,0
2,./ImageFolder/mty3.jpg,4.050000,0,1,1,0
3,./ImageFolder/mty4.jpg,3.316667,0,1,1,0
4,./ImageFolder/mty5.jpg,2.300000,0,1,1,0


In [10]:
df_female.shape

(120000, 4)

In [11]:
num_female = 2000
df_fty = get_median(num_female, df_female, "./ImageFolder/", "fty")

In [12]:
df_ftw.shape

(45000, 4)

In [40]:
df_ftw.head()
# df_ftw.loc[df_ftw['Rating']]

,Email,Filename,Rating,original Rating
0,1,ftw1.jpg,3,NaN
1,2,ftw1.jpg,4,5
2,3,ftw1.jpg,4,NaN
3,4,ftw1.jpg,5,NaN
4,5,ftw1.jpg,4,NaN


In [31]:
df_ftw_m = get_median(df_ftw.shape[0]//60, df_ftw, "./ImageFolder/", "ftw")

In [32]:
df_ftw_m.head()

,Filename,Rating,Female,Male,Yellow,White
0,./ImageFolder/ftw1.jpg,4.116667,1,0,0,1
1,./ImageFolder/ftw2.jpg,3.933333,1,0,0,1
2,./ImageFolder/ftw3.jpg,4.283333,1,0,0,1
3,./ImageFolder/ftw4.jpg,4.066667,1,0,0,1
4,./ImageFolder/ftw5.jpg,3.883333,1,0,0,1


In [33]:
df_mtw_m = get_median(df_mtw.shape[0]//60, df_mtw, "./ImageFolder/", "mtw")

In [35]:
df_mtw_m.head()

,Filename,Rating,Female,Male,Yellow,White
0,./ImageFolder/mtw1.jpg,3.116667,0,1,0,1
1,./ImageFolder/mtw2.jpg,3.700000,0,1,0,1
2,./ImageFolder/mtw3.jpg,4.183333,0,1,0,1
3,./ImageFolder/mtw4.jpg,4.316667,0,1,0,1
4,./ImageFolder/mtw5.jpg,2.666667,0,1,0,1


In [36]:
con = pd.concat([df_mty, df_fty, df_mtw_m, df_ftw_m])
con.head()

,Filename,Rating,Female,Male,Yellow,White
0,./ImageFolder/mty1.jpg,3.950000,0,1,1,0
1,./ImageFolder/mty2.jpg,3.600000,0,1,1,0
2,./ImageFolder/mty3.jpg,4.050000,0,1,1,0
3,./ImageFolder/mty4.jpg,3.316667,0,1,1,0
4,./ImageFolder/mty5.jpg,2.300000,0,1,1,0


In [37]:
con.tail()

,Filename,Rating,Female,Male,Yellow,White
745,./ImageFolder/ftw746.jpg,2.766667,1,0,0,1
746,./ImageFolder/ftw747.jpg,3.233333,1,0,0,1
747,./ImageFolder/ftw748.jpg,3.933333,1,0,0,1
748,./ImageFolder/ftw749.jpg,4.116667,1,0,0,1
749,./ImageFolder/ftw750.jpg,3.600000,1,0,0,1


In [38]:
con['Rating'].max(), con['Rating'].min()

(4.75, 1.0166666666666666)

In [39]:
con.to_csv("./median_include_white.csv", index=False)
con.head()

,Filename,Rating,Female,Male,Yellow,White
0,./ImageFolder/mty1.jpg,3.950000,0,1,1,0
1,./ImageFolder/mty2.jpg,3.600000,0,1,1,0
2,./ImageFolder/mty3.jpg,4.050000,0,1,1,0
3,./ImageFolder/mty4.jpg,3.316667,0,1,1,0
4,./ImageFolder/mty5.jpg,2.300000,0,1,1,0
